In [40]:
import torch
import torch.nn.functional as F
import sympy as sp
import pandas as pd
import numpy as np

In [3]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [18]:
df = pd.read_csv('physics_equations.csv')

functions = []
num_vars_per_func = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# variables']
    function_details = {
        'formula': formula,
        'variables': []
    }
    
    for i in range(1, 11):  
        v_name = row.get(f'v{i}_name', None)
        v_low = row.get(f'v{i}_low', None)
        v_high = row.get(f'v{i}_high', None)
        
        if pd.notna(v_name):
            function_details['variables'].append({
                'name': v_name,
                'low': v_low,
                'high': v_high
            })
    
    functions.append(function_details)
    num_vars_per_func.append(num_vars)

'''for i, func in enumerate(functions):
    print(f"Function {i+1}:")
    print(f"  Formula: {func['formula']}")
    print(f"  Number of Variables: {num_vars_per_func[i]}")
    print("  Variables:")
    for var in func['variables']:
        print(f"    - Name: {var['name']}, Range: ({var['low']}, {var['high']})")
    print()'''

'for i, func in enumerate(functions):\n    print(f"Function {i+1}:")\n    print(f"  Formula: {func[\'formula\']}")\n    print(f"  Number of Variables: {num_vars_per_func[i]}")\n    print("  Variables:")\n    for var in func[\'variables\']:\n        print(f"    - Name: {var[\'name\']}, Range: ({var[\'low\']}, {var[\'high\']})")\n    print()'

In [63]:
def generate_function(function, sample_size, x, max_vars, device):
    sympy_symbols = []
    param_tensors = []
    
    for var in function["variables"]:
        sym = sp.symbols(var["name"])
        sympy_symbols.append(sym)
        min_val, max_val = var["low"], var["high"]
        param = (max_val - min_val) * torch.rand(sample_size, 1, device=device) + min_val
        param_tensors.append(param)

    sympy_symbols.append(sp.symbols('x'))
    symbols = np.array(sympy_symbols)
    symbols = np.pad(symbols, (0, max_vars - len(symbols)))
    params = torch.cat(param_tensors, dim=1)
    padded_params = F.pad(params, pad=(0, max_vars - params.size(1)))
    padded_params = padded_params.expand(sample_size, max_vars)
    formula = sp.sympify(function["formula"])
    eval_func = sp.lambdify(sympy_symbols, formula, modules="numpy")
        
    results = []
    for xi in x:
        input_values = torch.cat([params, xi.expand(sample_size, 1)], dim=1)
        results.append(eval_func(*input_values.T))
    results = torch.stack(results, dim=1)
    
    return results, formula, symbols, padded_params

In [64]:
sample_size = 1000
sequence_length = 100
num_funcs = 10
max_vars = 5

x_values = torch.linspace(-1, 1, sequence_length).to(device)
hold = []
for f in functions[0:10]:
    try:
        results = generate_function(f, sample_size, x_values, max_vars, device)
        hold.append(results)
    except Exception as e:
        print(f"Error processing function {f}: {e}")

In [65]:
y_values = torch.stack([l[0] for l in hold])
formulas = [l[1] for l in hold]
symbols = [l[2] for l in hold]
param_values = torch.stack([l[3] for l in hold])
num_params = torch.tensor([len(l[2]) for l in hold])

num_params = torch.repeat_interleave(num_params, repeats=sample_size)
y_values = torch.flatten(y_values, end_dim=1)
param_values = torch.flatten(param_values, end_dim=1)
formulas = np.array([[formula for _ in range(1000)] for formula in formulas]).flatten()
symbols = np.array([[symbol for _ in range(1000)] for symbol in symbols])
symbols = symbols.reshape(num_funcs*sample_size, max_vars)

y_values.shape, param_values.shape, num_params.shape, formulas.shape, symbols.shape

(torch.Size([10000, 100]),
 torch.Size([10000, 5]),
 torch.Size([10000]),
 (10000,),
 (10000, 5))

In [69]:
y_values[2000][0], param_values[2000], num_params[2000], formulas[2000], symbols[2000]

(tensor(0.4813+0.j, device='cuda:4'),
 tensor([0.9626, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:4'),
 tensor(5),
 a*x**2/2,
 array([a, x, np.int64(0), np.int64(0), np.int64(0)], dtype=object))

In [8]:
def evaluate_function(params, symbols, formula, x):
    eval_func = sp.lambdify(symbols, formula, modules="numpy")
    sample_size = params.shape[0]
    results = []
    for xi in x:
        print(params)
        params_2d = params.view(sample_size, -1)
        xi_expanded = xi.expand(sample_size, 1)
        input_values = torch.cat([params_2d, xi_expanded], dim=1)        
        print(input_values)
        results.append(eval_func(*input_values.T))
    return torch.stack(results, dim=1)

In [9]:
def compute_derivatives(parameters, func, symbols, formula, x, epsilon=1e-6):
    batch_size = parameters.shape[0]
    num_params = parameters.shape[1]
    gradients = torch.zeros(batch_size, num_params)

    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(num_params):
            perturbed_params_pos = param_tensor.clone()
            perturbed_params_neg = param_tensor.clone()
            
            perturbed_params_pos[j] += epsilon
            print(perturbed_params_pos.shape)
            forward_values = func(perturbed_params_pos, symbols, formula, x)
            
            perturbed_params_neg[j] -= epsilon
            backward_values = func(perturbed_params_neg, symbols, formula, x)
            
            gradients[i, j] = (forward_values - backward_values) / (2 * epsilon)
    
    return gradients


In [10]:
def compute_hessians(parameters, func, symbols, formula, epsilon=1e-6):
    batch_size = parameters.shape[0]
    num_params = parameters.shape[2]
    hessian = torch.zeros(batch_size, num_params, num_params)

    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(num_params):
            for k in range(j, num_params):
                perturbed_params_pp = param_tensor.clone()
                perturbed_params_pp[0, j, 0] += epsilon
                perturbed_params_pp[0, k, 0] += epsilon
                ff_value = func(perturbed_params_pp, symbols, formula)

                perturbed_params_nn = param_tensor.clone()
                perturbed_params_nn[0, j, 0] -= epsilon
                perturbed_params_nn[0, k, 0] -= epsilon
                bb_value = func(perturbed_params_nn, symbols, formula)

                if j == k:
                    center_value = func(param_tensor, symbols, formula)
                    hessian[i, j, k] = (ff_value - 2 * center_value + bb_value) / (epsilon ** 2)
                else:
                    perturbed_params_pn = param_tensor.clone()
                    perturbed_params_pn[0, j, 0] += epsilon
                    perturbed_params_pn[0, k, 0] -= epsilon
                    fb_value = func(perturbed_params_pn, symbols, formula)

                    perturbed_params_np = param_tensor.clone()
                    perturbed_params_np[0, j, 0] -= epsilon
                    perturbed_params_np[0, k, 0] += epsilon
                    bf_value = func(perturbed_params_np, symbols, formula)

                    hessian[i, j, k] = (ff_value - fb_value - bf_value + bb_value) / (4 * epsilon ** 2)
                    hessian[i, k, j] = hessian[i, j, k]

    return hessian

In [51]:
x_values[0]

tensor(-1., device='cuda:4')

In [13]:
derivative_test = compute_derivatives(param_values[0], evaluate_function, hold_symbols[0], hold_formulas[0], x_values)
hessian_test = compute_hessians(param_values[0], evaluate_function, hold_symbols[0], hold_formulas[0])

torch.Size([3])
tensor([ 0.5163, -0.9938, -0.9851], device='cuda:4', grad_fn=<CopySlices>)
tensor([[ 0.5163, -1.0000],
        [-0.9938, -1.0000],
        [-0.9851, -1.0000]], device='cuda:4', grad_fn=<CatBackward0>)


TypeError: _lambdifygenerated() missing 2 required positional arguments: 'c' and 'x'

In [8]:
'''def compute_hessians(parameters, func, symbols, formula, epsilon=1e-6): #need to fix
    batch_size = parameters.shape[0]
    max_num_params = parameters.shape[2]
    hessians = torch.zeros((batch_size, max_num_params, max_num_params))
    
    for i in range(batch_size):
        param_tensor = parameters[i].clone().detach().requires_grad_(True)
        
        for j in range(max_num_params):
            for k in range(max_num_params):
                # Perturb j-th and k-th parameters
                perturbed_params = param_tensor.clone()
                
                # Compute f(x + epsilon * e_j + epsilon * e_k)
                perturbed_params_jk = perturbed_params.clone()
                perturbed_params_jk[:, j, :] += epsilon
                perturbed_params_jk[:, k, :] += epsilon
                f_plus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x + epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_plus_minus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j + epsilon * e_k)
                perturbed_params_jk[:, j, :] -= 2 * epsilon
                perturbed_params_jk[:, k, :] += 2 * epsilon
                f_minus_plus = func(perturbed_params_jk, symbols, formula)
                
                # Compute f(x - epsilon * e_j - epsilon * e_k)
                perturbed_params_jk[:, k, :] -= 2 * epsilon
                f_minus_minus = func(perturbed_params_jk, symbols, formula)
                hessians[i, j, k] = (f_plus_plus - f_plus_minus - f_minus_plus + f_minus_minus) / (4 * epsilon**2)
    
    return hessians'''

In [10]:
derivatives = [compute_derivative(param_values[i], evaluate_function, hold_symbols[i], hold_formulas[i]) for i in range(len(param_values))]
hessians = [compute_hessians(param_values[i], evaluate_function, hold_symbols[i], hold_formulas[i]) for i in range(len(param_values))]

In [42]:
flat_result = torch.flatten(torch.stack(y_values).unsqueeze(2), start_dim=0, end_dim=1)

max_der = max(d.size(1) for d in derivatives)
padded_ders = []
for d in derivatives:
    padded_tensor = F.pad(d, pad=(0, max_der - d.size(1)))
    padded_ders.append(padded_tensor)
flat_der = torch.flatten(torch.stack(padded_ders), start_dim=0, end_dim=1)

max_hess = max_der**2
padded_hess = []
for h in hessians:
    h = torch.flatten(h, start_dim=1)
    padded_tensor = F.pad(h, pad=(0, max_hess - h.size(1)))
    padded_hess.append(padded_tensor)
flat_hess = torch.flatten(torch.stack(padded_hess), start_dim=0, end_dim=1)

flat_der = F.pad(flat_der, (0, flat_hess.size(-1) - flat_der.size(-1)))
flat_result = F.pad(flat_result, (0, flat_hess.size(-1) - flat_result.size(-1)))

flat_result.shape, flat_der.shape, flat_hess.shape

(torch.Size([10000, 81]), torch.Size([10000, 81]), torch.Size([10000, 81]))

In [44]:
torch.save({
    'flattened_data': torch.stack([flat_result, flat_der, flat_hess], dim=2),
    'results': y_values,
    'formulas': hold_formulas,
    'symbols': hold_symbols,
    'params': param_values,
    'num_params': num_params,
}, 'hold_data.pth')